In [2]:
# from   pathlib      import Path
# import importlib

# import numpy                as np
# import matplotlib           as mpl
# import matplotlib.pyplot    as plt
# import seaborn              as sns
# import pandas               as pd

# from scipy.stats   import kruskal, wilcoxon, mannwhitneyu, ranksums
# from scipy.optimize import curve_fit
# import statsmodels.api as sm
# from statsmodels.multivariate.manova import MANOVA
# import statsmodels.formula.api as smf
# from eidynamics     import utils, plot_tools
# plt.rcParams['font.family'] = 'Arial'
# plt.rcParams['font.size'] = 12
# plt.rcParams['svg.fonttype'] = 'none'

# # make a colour map viridis
# viridis = mpl.colormaps["viridis"]
# flare   = mpl.colormaps["flare"]
# crest   = mpl.colormaps["crest"]
# magma   = mpl.colormaps["magma"]
# edge    = mpl.colormaps['edge']

# color_E = "flare"
# color_I = "crest"
# color_freq = {1:magma(0.05), 5:magma(0.1), 10:magma(0.2), 20:magma(.4), 30:magma(.5), 40:magma(.6), 50:magma(.7), 100:magma(.9)}
# color_squares = color_squares = {1:viridis(0.2), 5:viridis(.4), 7:viridis(.6), 15:viridis(.8), 20:viridis(1.0)}
# color_EI = {-70:flare(0), 0:crest(0)}
# colors_EI = {-70:flare, 0:crest}

# Fs = 2e4
# %matplotlib widget
# freq_sweep_pulses = np.arange(9)

# from eidynamics.fit_PSC     import find_sweep_expected
# # from Findsim        import tab_presyn_patterns_LR_43
# # import parse_data
# from eidynamics     import utils, plot_tools
# import all_cells
# # import plotFig2
# import stat_annotate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import plotFig2
%colors nocolor
Fs=2e4

In [4]:
# load data
figure_raw_material_location = Path(r"paper_figure_matter\\")
# paper_figure_export_location = Path(r"paper_figures\\Figure2v2\\")
data_path                    = Path(r"parsed_data\\FreqSweep\\")

# Load the dataset
freq_sweep_vc_datapath =  data_path / "all_cells_FreqSweep_VC_long.h5"
df = pd.read_hdf(freq_sweep_vc_datapath, key='data')

# # expanded dataframe (processed dataframe with metadata and analysed params)
# expanded_data_path = Path(r"parsed_data\all_cells_FreqSweep_combined_expanded.h5")
# xc_FS_analyseddf = pd.read_hdf(expanded_data_path, key='data')

In [5]:
df.shape

(4407, 80073)

In [6]:
# remove columns that are not needed
# not needed columns are 20049 to 80049
df = df.drop(df.columns[20049:80049], axis=1)
df.shape

(4407, 20073)

In [7]:
import importlib
importlib.reload(plotFig2)

new_fields = ['valley_', 'valleytime_', 'peak_', 'peaktime_', 'slope_', 'delay_', 'normpeak_', 'PSC_', 'normPSC_', 'spike_']
newcolumns = [field + str(i) for field in new_fields for i in range(9)]
Fs=2e4
# Adding new columns with NaN values
df = df.assign(**{col: np.nan for col in newcolumns})
c = 0
d=0
r=0
k=0
for idx, row in df.iterrows():
    print(f"Processing {idx} row with trialID {row['trialID']}")
    # do not process if freq==10
    if row['stimFreq'] <20:
        print(f"Skipping {idx} row with trialID {row['trialID']}: Low frequency")
        d+=0
        continue
    
    try:
        if row['probePulseStart'] == row['pulseTrainStart']:
            freq = row['stimFreq']
            isi = int(0.9*Fs / freq)
            print(f"interpolating {idx} row with trialID {row['trialID']}: No probe pulse. {row['probePulseStart']} =/= { row['pulseTrainStart']}")
            row[2000:2000+isi] = row[4620:4620+isi]
            row[2000+isi-100:4620] = np.interp(np.arange(2000+isi-100,4620), [2000+isi-100,4620], [row[2000+isi-100], row[4620]])
            c+=1
            results = plotFig2.deconv(row[49:80049], row['stimFreq'], 0.1, row['pulseTrainStart'], None, noprobepulse=False)
        else:
            results = plotFig2.deconv(row[49:80049], row['stimFreq'], row['probePulseStart'], row['pulseTrainStart'], None, noprobepulse=False)
    except RuntimeError:
        r+=1
        continue
    except FloatingPointError as e:
        print(f"Error at {idx} row with trialID {row['trialID']}: {e}")
        print(row['stimFreq'], row['numSq'], row['probePulseStart'], row['pulseTrainStart'])
        k+=1
        continue
    
    valleyTimes, valleys, peakTimes, peaks = results[2]
    # elementwise difference between peak and valley is PSC
    pscs = np.array(peaks) - np.array(valleys)
    normPSCs = np.array(pscs) / pscs[0]
    normpeaks = np.array(peaks) / peaks[0]
    stimTimes = np.array(results[-1]) / Fs    
    slopes = np.array([(peaks[i] - valleys[i]) / (peakTimes[i] - valleyTimes[i]) for i in range(len(peaks))])
    delays = np.array([(valleyTimes[i] - stimTimes[i]) for i in range(len(valleyTimes))])
    # spikes if peaks are greater than 20
    spikes = np.array([1 if peak > 20 else 0 for peak in peaks])

    for col in newcolumns:
        prefix, col_idx = col.split('_')
        col_idx = int(col_idx)

        if col_idx < len(valleys):  # Ensure index is within bounds
            if prefix == 'valley':
                df.loc[row.name, col] = valleys[col_idx]
            elif prefix == 'valleytime':
                df.loc[row.name, col] = valleyTimes[col_idx]
            elif prefix == 'peak':
                df.loc[row.name, col] = peaks[col_idx]
            elif prefix == 'peaktime':
                df.loc[row.name, col] = peakTimes[col_idx]
            elif prefix == 'slope':
                df.loc[row.name, col] = slopes[col_idx]
            elif prefix == 'delay':
                df.loc[row.name, col] = delays[col_idx]
            elif prefix == 'normpeak':
                df.loc[row.name, col] = normpeaks[col_idx]
            elif prefix == 'PSC':
                df.loc[row.name, col] = pscs[col_idx]
            elif prefix == 'normPSC':
                df.loc[row.name, col] = normPSCs[col_idx]
            elif prefix == 'spike':
                df.loc[row.name, col] = spikes[col_idx]


    # counter for every 100 rows
    if idx % 100 == 0:
        print(f"Processed {idx} rows")

# Assuming df is your final DataFrame
metadata_cols1 = df.iloc[:, :49]
metadata_cols2 = df.iloc[:, 20049:]

# Concatenating the two slices into a new DataFrame
new_df = pd.concat([metadata_cols1, metadata_cols2], axis=1)

# Display the shape of the new DataFrame to verify
print(new_df.shape)
print(f"Kernel errors: {k}, Runtime errors: {r}, No probe pulse: {c}, Low frequency: {d}")

Processing 0 row with trialID 1931_0_1
negative kernel!
Error at 0 row with trialID 1931_0_1: calcKernel: negative kernel
20 5 0.20000000298023224 0.5
Processing 1 row with trialID 1931_0_2
negative kernel!
Error at 1 row with trialID 1931_0_2: calcKernel: negative kernel
20 5 0.20000000298023224 0.5
Processing 2 row with trialID 1931_0_3
negative kernel!
Error at 2 row with trialID 1931_0_3: calcKernel: negative kernel
20 5 0.20000000298023224 0.5
Processing 3 row with trialID 1931_0_4
negative kernel!
Error at 3 row with trialID 1931_0_4: calcKernel: negative kernel
20 5 0.20000000298023224 0.5
Processing 4 row with trialID 1931_0_5
negative kernel!
Error at 4 row with trialID 1931_0_5: calcKernel: negative kernel
20 5 0.20000000298023224 0.5
Processing 5 row with trialID 1931_0_6
negative kernel!
Error at 5 row with trialID 1931_0_6: calcKernel: negative kernel
20 15 0.20000000298023224 0.5
Processing 6 row with trialID 1931_0_7
negative kernel!
Error at 6 row with trialID 1931_0_7:

c:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Analysis\plotFig2.py:48: RuntimeWarning: overflow encountered in exp
  ret, cov = sci.curve_fit(lambda t,a,tau: a*np.exp(-t/tau), x, y, p0=(pk-baseline,0.02) )
c:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Analysis\plotFig2.py:48: RuntimeWarning: overflow encountered in multiply
  ret, cov = sci.curve_fit(lambda t,a,tau: a*np.exp(-t/tau), x, y, p0=(pk-baseline,0.02) )


Processing 358 row with trialID 1541_3_23
Processing 359 row with trialID 1541_3_24
Processing 360 row with trialID 1541_4_1
Processing 361 row with trialID 1541_4_2
Processing 362 row with trialID 1541_4_3
Processing 363 row with trialID 1541_4_4
Processing 364 row with trialID 1541_4_5
Processing 365 row with trialID 1541_4_6
Processing 366 row with trialID 1541_4_7
Processing 367 row with trialID 1541_4_8
Processing 368 row with trialID 1541_4_9
Processing 369 row with trialID 1541_4_10
Processing 370 row with trialID 1541_4_11
Processing 371 row with trialID 1541_4_12
Processing 372 row with trialID 1541_4_13
Processing 373 row with trialID 1541_4_14
Processing 374 row with trialID 1541_4_15
Processing 375 row with trialID 1541_4_16
Processing 376 row with trialID 1541_4_17
Processing 377 row with trialID 1541_4_18
Processing 378 row with trialID 1541_4_19
Processing 379 row with trialID 1541_4_20
Processing 380 row with trialID 1541_4_21
Processing 381 row with trialID 1541_4_22
P

c:\venvs\venv-ephys\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Processing 756 row with trialID 1531_12_13
Processing 757 row with trialID 1531_12_14
Processing 758 row with trialID 1531_12_15
Processing 759 row with trialID 1531_12_16
Processing 760 row with trialID 1531_12_17
Processing 761 row with trialID 1531_12_18
Processing 762 row with trialID 1531_12_19
Processing 763 row with trialID 1531_12_20
Processing 764 row with trialID 1531_12_21
Processing 765 row with trialID 1531_12_22
Processing 766 row with trialID 1531_12_23
Processing 767 row with trialID 1531_12_24
Processing 768 row with trialID 1531_13_1
Processing 769 row with trialID 1531_13_2
Processing 770 row with trialID 1531_13_3
Processing 771 row with trialID 1531_13_4
Processing 772 row with trialID 1531_13_5
Processing 773 row with trialID 1531_13_6
Processing 774 row with trialID 1531_13_7
Processing 775 row with trialID 1531_13_8
Processing 776 row with trialID 1531_13_9
Processing 777 row with trialID 1531_13_10
Processing 778 row with trialID 1531_13_11
Processing 779 row w

In [8]:
print(new_df.shape)
# save df
new_df.to_hdf(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements.h5", key='data', mode='w')
# save as excel
new_df.to_excel(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements.xlsx")

# drop those rows from df that have NaN values in column 'valley_0'
new_df = new_df.dropna(subset=['valley_0'])
print(new_df.shape)


(4407, 163)


C:\Users\adity\AppData\Local\Temp\ipykernel_80396\660631201.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->Index(['sex', 'unit', 'location', 'protocol', 'numSq', 'clampMode',
       'clampPotential', 'condition', 'numPatterns', 'patternList',
       'frameChangeTimes', 'pulseTimes', 'fieldData', 'peaks_cell',
       'peaks_cell_norm', 'auc_cell', 'slope_cell', 'delay_cell',
       'peaks_field', 'peaks_field_norm', 'cell_fpr', 'field_fpr', 'cell_ppr',
       'cell_stpr', 'field_ppr', 'field_stpr', 'trialID', 'cellunit',
       'fieldunit'],
      dtype='object')]

  new_df.to_hdf(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements.h5", key='data', mode='w')


(2581, 163)


In [9]:
# save df
new_df.to_hdf(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements_noNANs.h5", key='data', mode='w')
# save as excel
new_df.to_excel(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements_noNANs.xlsx")

C:\Users\adity\AppData\Local\Temp\ipykernel_80396\1705529343.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->Index(['sex', 'unit', 'location', 'protocol', 'numSq', 'clampMode',
       'clampPotential', 'condition', 'numPatterns', 'patternList',
       'frameChangeTimes', 'pulseTimes', 'fieldData', 'peaks_cell',
       'peaks_cell_norm', 'auc_cell', 'slope_cell', 'delay_cell',
       'peaks_field', 'peaks_field_norm', 'cell_fpr', 'field_fpr', 'cell_ppr',
       'cell_stpr', 'field_ppr', 'field_stpr', 'trialID', 'cellunit',
       'fieldunit'],
      dtype='object')]

  new_df.to_hdf(data_path / "all_cells_FreqSweep_VC_kernelfit_response_measurements_noNANs.h5", key='data', mode='w')
